##### 【 모델 성능 평가 지표들 - 분류용 】

[1] 모듈 로딩 및 데이터 준비<hr>

In [ ]:
## [1-1] 모듈 로딩
##-> 기본 모듈
import numpy as np 
import pandas as pd 

##-> ML 관련 모듈
## 학습 알고리즘 모듈
from sklearn.neighbors import KNeighborsClassifier  

## 성능 평가관련 모듈
from sklearn.metrics   import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics   import classification_report

## 학습 준비관련 모듈
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import LabelEncoder 

In [5]:
## [1-2] 데이터 준비
DATA_FILE = '../Data/iris.csv'

irisDF = pd.read_csv(DATA_FILE)
irisDF.head(3)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa


[2] 데이터 학습 준비<hr>

In [ ]:
## [2-1] 피쳐와 타겟, 학습용과 테스트용 데이터셋 분리
##-> 피쳐와 타겟 분리
featureDF = irisDF[irisDF.columns[:-1]]
targetSR  = irisDF[irisDF.columns[-1]]
print(f'featureDF : {featureDF.shape},  targetSR :{targetSR.shape}')


(150, 4),  (150,)


In [67]:
##-> 학습용/테스트용 데이터셋 분리
x_train, x_test, y_train, y_test = train_test_split(featureDF, 
                                                     targetSR,
                                                     test_size = 0.2,
                                                     random_state = 42,
                                                     stratify=targetSR)

print(f'[TRAIN] : {x_train.shape},  {y_train.shape}')
print(f'[TEST ] : {x_test.shape},   {y_test.shape}')

[TRAIN] : (120, 4),  (120,)
[TEST ] : (30, 4),   (30,)


In [68]:
## [2-2] 타겟 컬럼 수치화 : LabelEncoder
lbEncoder       = LabelEncoder()

encode_y_train = lbEncoder.fit_transform(y_train)
encode_y_test  = lbEncoder.transform(y_test)

print(f'classes_ : {lbEncoder.classes_}')

classes_ : ['Setosa' 'Versicolor' 'Virginica']


In [69]:
## [2-3] 수치형 피쳐 컬럼 : 스케일링
from sklearn.preprocessing import RobustScaler, StandardScaler

#-> 평균 0, 분산 1이 되도록 스케일링 : 이상치 민감
stdScaler   = StandardScaler()
std_x_train = stdScaler.fit_transform(x_train)
std_x_test  = stdScaler.transform(x_test)

#-> 중간/중앙값으로 스케일링 : 이상치 강함
rbScaler   = RobustScaler()
rb_x_train = rbScaler.fit_transform(x_train)
rb_x_test  = rbScaler.transform(x_test)


[3] 학습 <hr>

In [70]:
## [3-1] GridSearchCV기반 교차검증 진행 => 일반화 성능 체크
from sklearn.model_selection import GridSearchCV 

## 필요준비 : 모델 인스턴스, 모델의 하이퍼파라미터 dict 
knn    = KNeighborsClassifier()
params = {'n_neighbors':[3,5,7]}

## 교차 검증 및 최적의 하이퍼파라미터 찾기 진행
search = GridSearchCV(knn, 
                      params, 
                      return_train_score=True, 
                      scoring='f1_micro')

## => 학습용 데이터셋으로 교차검증 진행 => 데이터 누수 (CV)
search.fit(std_x_train, encode_y_train)

## => 결과 
print('cv_results_',  search.cv_results_.keys())
print('best_score_',  search.best_score_)
print('best_params_', search.best_params_)


cv_results_ dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_n_neighbors', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'split3_train_score', 'split4_train_score', 'mean_train_score', 'std_train_score'])
best_score_ 0.9666666666666668
best_params_ {'n_neighbors': 5}


In [71]:
## [3-2] 최적의 하이퍼파라미터 적용 + 전체 학습데이터로 학습 진행
BEST_N = search.best_params_['n_neighbors']

final_model = KNeighborsClassifier(n_neighbors= BEST_N)
final_model.fit(std_x_train, encode_y_train)

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


[4] 다양한 분류용 성능지표기반 평가<hr>

In [72]:
## 성능지표함수(1D_정답, 1D_예측)
pre_y = final_model.predict(std_x_test)

##- 성능지표 : 정확도
acc_1 = accuracy_score(encode_y_test, pre_y)
acc_2 = final_model.score(std_x_test, encode_y_test)
print(f'accuracy_score() : {acc_1},  score() 메서드 :{acc_2}')

accuracy_score() : 0.9333333333333333,  score() 메서드 :0.9333333333333333


In [73]:
##- 성능지표 : 정밀도 => 모델 예측값 중심 
precision_ = precision_score(encode_y_test, pre_y, average='micro')
print(f'precision_score() : {precision_}')

precision_score() : 0.9333333333333333


In [74]:
##- 성능지표 : 재현률 => 정답/타겟 중심
recall_ = recall_score(encode_y_test, pre_y, average='micro')
print(f'recall_score() : {recall_}')

recall_score() : 0.9333333333333333


In [75]:
##- 성능지표 : f1점수 => 정밀도와 재현율의 차이 해결위해서 조화평균
f1_ = f1_score(encode_y_test, pre_y, average='micro')
print(f'f1_score() : {f1_}')

f1_score() : 0.9333333333333333


In [76]:
##- 분류 보고서 : 모든 성능지표 및 평균 계산 방식 적용한 보고서
result = classification_report(encode_y_test, pre_y)
print(result)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.83      1.00      0.91        10
           2       1.00      0.80      0.89        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30

